# 5th attempt - RNN

In [65]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [66]:
SIZE = 10
AMOUNT_BOARDS = 1000

In [67]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [68]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [69]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 41366
reverse df sort: 41366


In [70]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  33506
len x test:  4137
len y train:  33506
len y test:  4137


In [71]:
X_train.shape

(33506, 100)

In [72]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [73]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 100)
(33506,)


In [74]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [75]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 10, 10, 1)
(33506, 1)


In [76]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [77]:
import tensorflow as tf

# הנחה: הנתונים שלך כבר בטעינה בשם:
# X_train_array: (33506, 10, 10, 1)
# y_train_array: (33506, 1)

INPUT_DIM = SIZE * SIZE  
TIMESTEPS = gen-1

# שינוי צורת הנתונים כך שתתאים ל-LSTM
X_train = X_train_array.reshape((-1, TIMESTEPS, INPUT_DIM)).astype('float32')
y_train = y_train_array.astype('float32')

print("Reshaped X_train:", X_train.shape)
print("y_train:", y_train.shape)

# הגדרת המודל
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(TIMESTEPS, INPUT_DIM)),
    tf.keras.layers.LSTM(128, activation='tanh'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # סיווג בינארי
])


Reshaped X_train: (33506, 1, 100)
y_train: (33506, 1)


In [78]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 125,569 (490.50 KB)

 Trainable params: 125,569 (490.50 KB)

 Non-trainable params: 0 (0.00 B)

In [79]:
# אימון
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.7988 - loss: 0.4521 - val_accuracy: 0.8121 - val_loss: 0.4001
Epoch 2/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.8178 - loss: 0.3791 - val_accuracy: 0.8133 - val_loss: 0.3871
Epoch 3/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8290 - loss: 0.3591 - val_accuracy: 0.8218 - val_loss: 0.3760
Epoch 4/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.8443 - loss: 0.3342 - val_accuracy: 0.8275 - val_loss: 0.3678
Epoch 5/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8604 - loss: 0.3066 - val_accuracy: 0.8320 - val_loss: 0.3672
Epoch 6/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8674 - loss: 0.2944 - val_accuracy: 0.8302 - val_loss: 0.3802
Epoch 7/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.8785 - loss: 0.2734 - val_accuracy: 0.8251 - val_loss: 0.3892
Epoch 8/20
838/838 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8960 - loss: 0.2450 - va

In [80]:
import numpy as np

def evaluate_model_detailed(model, X_test, y_test, threshold=0.5):
    """
    מחשבת מדדים מפורטים לרשת Game of Life:
    כמה תאים חיים נחזו כמתים ולהפך, כולל Accuracy, Precision, Recall ו-F1.

    Args:
        model: מודל מאומן של Keras
        X_test: מערך קלט לבדיקה
        y_test: פלט אמיתי לבדיקה
        threshold: סף בינאריזציה (ברירת מחדל 0.5)
    """
    # חיזוי
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob >= threshold).astype(int)
    y_true = y_test.astype(int)

    # flatten להשוואה תא-תא
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # חישוב קטגוריות
    TP = np.sum((y_true_flat == 1) & (y_pred_flat == 1))
    TN = np.sum((y_true_flat == 0) & (y_pred_flat == 0))
    FP = np.sum((y_true_flat == 0) & (y_pred_flat == 1))
    FN = np.sum((y_true_flat == 1) & (y_pred_flat == 0))

    total_cells = len(y_true_flat)
    accuracy = (TP + TN) / total_cells
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # הדפסה מסודרת
    print("===== Evaluation Results =====")
    print(f"Total cells: {total_cells}")
    print(f"True Positives (alive predicted alive): {TP}")
    print(f"True Negatives (dead predicted dead): {TN}")
    print(f"False Positives (dead predicted alive): {FP}")
    print(f"False Negatives (alive predicted dead): {FN}")
    print("--------------------------------")
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1_score:.4f}")
    print("================================")

    return {
        "TP": TP, "TN": TN, "FP": FP, "FN": FN,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }


In [82]:
# הכנת X_test ו-y_test
X_test = X_test_array.reshape((-1, gen-1, SIZE, SIZE, 1)).astype('float32')
y_test = y_test_array.astype('float32')

# בדיקה מפורטת
results = evaluate_model_detailed(model, X_test, y_test)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 1, 10, 10, 1), dtype=float32). Expected shape (None, 1, 100), but input has incompatible shape (32, 1, 10, 10, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 1, 10, 10, 1), dtype=float32)
  • training=False
  • mask=None

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def evaluate_model_detailed2(model, X_test, y_test, threshold=0.5, show_example=True):
    """
    מחשבת מדדים מפורטים לרשת Game of Life ומציגה heatmap של טעויות.
    
    Args:
        model: מודל מאומן של Keras
        X_test: מערך קלט לבדיקה
        y_test: פלט אמיתי לבדיקה
        threshold: סף בינאריזציה (ברירת מחדל 0.5)
        show_example: האם להציג דוגמה גרפית של השגיאות
    """
    # --- חיזוי ---
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob >= threshold).astype(int)
    y_true = y_test.astype(int)

    # flatten להשוואה תא-תא
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # --- חישוב קטגוריות ---
    TP = np.sum((y_true_flat == 1) & (y_pred_flat == 1))
    TN = np.sum((y_true_flat == 0) & (y_pred_flat == 0))
    FP = np.sum((y_true_flat == 0) & (y_pred_flat == 1))
    FN = np.sum((y_true_flat == 1) & (y_pred_flat == 0))

    total_cells = len(y_true_flat)
    accuracy = (TP + TN) / total_cells
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # --- הדפסה ---
    print("===== Evaluation Results =====")
    print(f"Total cells: {total_cells}")
    print(f"True Positives (alive predicted alive): {TP}")
    print(f"True Negatives (dead predicted dead): {TN}")
    print(f"False Positives (dead predicted alive): {FP}")
    print(f"False Negatives (alive predicted dead): {FN}")
    print("--------------------------------")
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1_score:.4f}")
    print("================================")

    # --- הצגה גרפית ---
    if show_example:
        idx = np.random.randint(0, len(X_test))
        y_t = y_true[idx].squeeze()
        y_p = y_pred[idx].squeeze()

        diff_map = np.zeros_like(y_t, dtype=int)
        diff_map[(y_t == 0) & (y_p == 1)] = 1   # FP → אדום
        diff_map[(y_t == 1) & (y_p == 0)] = -1  # FN → כחול

        plt.figure(figsize=(12,4))

        plt.subplot(1,3,1)
        plt.title("True Board")
        plt.imshow(y_t, cmap='gray')
        plt.axis('off')

        plt.subplot(1,3,2)
        plt.title("Predicted Board")
        plt.imshow(y_p, cmap='gray')
        plt.axis('off')

        plt.subplot(1,3,3)
        plt.title("Errors (Red=FP, Blue=FN)")
        plt.imshow(diff_map, cmap='bwr', vmin=-1, vmax=1)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    return {
        "TP": TP, "TN": TN, "FP": FP, "FN": FN,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }


In [ ]:
results = evaluate_model_detailed2(model, X_test, y_test)